In [2]:
import pandas as pd
## get df from hugging face, same as HumanEvalPlusOrig.jsonl
df = pd.read_parquet("hf://datasets/evalplus/humanevalplus/data/test-00000-of-00001-5973903632b82d40.parquet")

In [3]:
df

,task_id,prompt,canonical_solution,entry_point,test
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,\n\n sorted_numbers = sorted(numbers)\n ...,has_close_elements,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,"\n\n cnt, group, results = 0, """", []\n f...",separate_paren_groups,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,\n\n return number - int(number)\n\n,truncate_number,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,\n account = 0\n for operation in operat...,below_zero,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,\n mean = sum(numbers) / len(numbers)\n ...,mean_absolute_deviation,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
...,...,...,...,...,...
159,HumanEval/159,"\ndef eat(number, need, remaining):\n """"""\n...",\n\n if need <= remaining:\n return ...,eat,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
160,HumanEval/160,"\ndef do_algebra(operator, operand):\n """"""\...","\n\n exp = """"\n for i in range(len(opera...",do_algebra,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
161,HumanEval/161,"\ndef solve(s):\n """"""You are given a string...","\n ans, has_letter = """", False\n for ch ...",solve,\n\nimport numpy as np\n\ndef is_floats(x) -> ...
162,HumanEval/162,"\ndef string_to_md5(text):\n """"""\n Given...","\n if text == """": return None\n import h...",string_to_md5,\n\nimport numpy as np\n\ndef is_floats(x) -> ...


## Instructions

1. Modify `HumanEvalPlusOrig` into executable test code.
2. Use `cProfile` to measure the runtime.

## Future Work

- Compare the computation time of the generated improved code with the `canonical_solution`.
- Use `perf` (Linux required) for detailed analysis (cirron failed, maybe someone else can try).

In [6]:
import pstats
import cProfile
results = pd.DataFrame(columns=['task_id', 'entry_point', 'total_time_microseconds'])

def create_executable_script(row, output_filename):
    with open(output_filename, 'w',encoding='utf-8') as f:
        f.write(row['prompt'] + '\n')
        
        solution = row['canonical_solution'].strip().split('\n')
        flag = 0
        for line in solution:
            if flag ==0:
                f.write('    ' + line + '\n')
                flag=1
            else:
                f.write(line + '\n')
        
        f.write('\n')  
        
        f.write(row['test'] + '\n')
        
        f.write('if __name__ == "__main__":\n')
        f.write(f'    check({row["entry_point"]})\n')

for i in range(len(df)):
    row = df.iloc[i]
    task_id = row['task_id']
    entry_point = row['entry_point']
    create_executable_script(row, 'test_code.py')

    def run_time():
        import test_code
    cProfile.run('run_time()', 'output.prof')
    p = pstats.Stats('output.prof')

    total_time_seconds = p.total_tt
    

    total_time_microseconds = total_time_seconds * 1_000_000
    
    temp_df = pd.DataFrame({
        'task_id': [task_id],
        'entry_point': [entry_point],
        'total_time_microseconds': [total_time_microseconds]
    })

    results = pd.concat([results, temp_df], ignore_index=True)

In [7]:
results

,task_id,entry_point,total_time_microseconds
0,HumanEval/0,has_close_elements,19.8
1,HumanEval/1,separate_paren_groups,25.6
2,HumanEval/2,truncate_number,25.9
3,HumanEval/3,below_zero,21.7
4,HumanEval/4,mean_absolute_deviation,23.7
...,...,...,...
159,HumanEval/159,eat,18.0
160,HumanEval/160,do_algebra,17.6
161,HumanEval/161,solve,18.6
162,HumanEval/162,string_to_md5,19.0
